# Programming Flow
## Initialize Session:
Opens communication with device. 

Relevant  c function : *niHSDIO_InitGenerationSession*

http://zone.ni.com/reference/en-XX/help/370520P-01/hsdiocref/cvinihsdio_initgenerationsession/ 

## Select Channels:
Assign either dynamic or static channels:
Any mix of dynamic/static generation and dynamic/static acquisition is good. 

__Def Static generation__ : 
Static generation places a single pattern on the configured channels. Static generation, like static acquisition, is controlled by software and does not use hardware timing.

Because a function call is required for each data point generated, static generation is generally used only for single-point or low-speed applications. Static generation can be helpful in system and cable debugging, DC-level semiconductor testing, and many other applications.

Static generation is also called immediate, unstrobed, or nonlatched generation.

__Def Dynamic generation__ : 
Dynamic generation is a clocked operation where binary data is sent from the NI device to the DUT (device under test, I belive this is the black box that serves as a breakout) across multiple digital channels. The data can be generated based on complex scripts, and it can react to triggers, generate markers, and be shifted in time with respect to the generating clock. 

For NI 6541/6542 devices Dynamic Generation is only supported in Active Drive mode.

__Conclusion__ From the above, it seems we want to set up our output channels to use dynamic generation, so we can script the timing, rather than rely on precise timing in python. It would apprear that this can be best implemented using "scripts" (as specifically defined in the NI documentation) for our use case.

relevant c function : 
*niHSDIO_AssignDynamicChannels*
http://zone.ni.com/reference/en-XX/help/370520P-01/hsdiocref/cvinihsdio_assigndynamicchannels/

## Configure Hardware:
Use Configuration VIs and functions to set up the triggers, voltage levels, and other settings and features needed for your data operation.

### Configure Sample Clock
niHSDIO_ConfigureSampleClock
The Sample clock is the primary timebase for the digital waveform generator/analyzer. This clock controls the rate at which samples are acquired or generated. Each period of the Sample clock is capable of initiating the acquisition or generation of one sample per channel.

**Options**:
* OnBoardClock : On board clock
* ClkIn : Use signal at front panel CLK IN connector
* PXI_STAR : use PXI_STAR ignal on the PXI backplane, usable only by PXI devices other than those in Slot 2
* STROBE : Uses clock present at STROBE channel of the DDC connector. Only valid for acquisition sessions

## Waveforms
Multiple waveforms can be saved to NI device memory (64ms?). We may be able to write our waveforms to this memory directly using one of the *niHISDIO_WriteNamedWaveform___* functions to write the waveforms from PC memory to device memory.

If our device memory is too small to support our current functional waveforms (or some waveforms some amount larger) we'll try to make use of streaming the waveforms.

In [1]:
import numpy as np
import os
from ctypes import *

In [2]:
ivi32_path = "C:\Program Files (x86)\IVI Foundation\IVI"
ivi32_bin = os.path.join(ivi32_path,"Bin")
ivi32_include = os.path.join(ivi32_path,"Include")

In [3]:
hsdio = CDLL(os.path.join(ivi32_bin,"niHSDIO.dll"))

In [4]:
hsdio.NIHSDIO_VAL_ON_BOARD_CLOCK_STR

AttributeError: function 'NIHSDIO_VAL_ON_BOARD_CLOCK_STR' not found

In [11]:
dev_adr = "Dev1"
cdev_adr = c_char_p(dev_adr.encode('utf-8'))
print(cdev_adr.value)
id_q = c_bool(1)
rst_int = c_bool(1)
opt_str = c_char_p("".encode('utf-8'))
vi = c_int(0)
err_c = hsdio.niHSDIO_InitGenerationSession(cdev_adr,id_q,rst_int,opt_str,byref(vi))

b'Dev1'


In [15]:
print(err_c )
print(type(err_c) )
print(c_int32(err_c) )

-1073807343
<class 'int'>
c_long(-1073807343)


In [ ]:
import struct
print(struct.calcsize("P")*8)

In [ ]:
class vi_int32():
    def __init__(val):
        self.act = c_int32(val)
        self.value = self.act.value
    

In [ ]:
help(c_int)